In [1]:
import tensorflow as tf
import numpy as np
from util import *
import os
import matplotlib.pyplot as plt
from tensorflow.python.ops import control_flow_ops
from losses import *
from flags_and_variables import *
from encode_boxes import *
from fcos import *
from tensorpack.tfutils.optimizer import AccumGradOptimizer

In [2]:
pyramid_dict = get_fpn_output(inputs,is_training=True,is_trainable=True)
centerness_output,classes_output,boxes_output = get_network_output(feature_layer_list,pyramid_dict,feature_size)
centerness_output = tf.concat(centerness_output,axis=1)
classes_output = tf.concat(classes_output,axis=1)
boxes_output = tf.concat(boxes_output,axis=1)

In [3]:
###testing part
centerness_output_test = tf.sigmoid(centerness_output)
classes_output_test = tf.sigmoid(classes_output)
boxes_output_test = boxes_output

In [4]:
#focal loss
focal_losses = focal_loss(classes,classes_output_test,state)
IOU_losses = iou_loss(boxes, boxes_output_test,state,centerness)
centerness_losses = centerness_loss(centerness, centerness_output_test,state)
regular_loss = tf.losses.get_regularization_loss()
total_losses = focal_losses+IOU_losses+centerness_losses+regular_loss

In [5]:
tf.summary.scalar('scala/total_losses',total_losses)
tf.summary.scalar('scala/focal_losses',focal_losses)
tf.summary.scalar('scala/IOU_losses',IOU_losses)
tf.summary.scalar('scala/centerness_losses',centerness_losses)
tf.summary.scalar('scala/reg_loss',regular_loss)
ckpt = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
if ckpt:
    start_step = int(ckpt.split('-')[-1])
else:
    start_step = 0  
    
with tf.variable_scope("optimizer") as scope:
    global_step = tf.get_variable("step", [], initializer=tf.constant_initializer(start_step*5), trainable=False)
    rate = tf.train.exponential_decay(FLAGS.learning_rate, global_step, decay_steps=20000, decay_rate=0.1, staircase=True)
    update_opts = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies([tf.group(update_opts)]):
        adam_opt = tf.train.AdamOptimizer(rate)
        train_opt = AccumGradOptimizer(adam_opt, niter=5).minimize(total_losses,global_step=global_step)
tf.summary.scalar('scala/learning_rate',rate)
merged_summary_op = tf.summary.merge_all()

[1213 14:36:47 @gradproc.py:79] WRN No gradient w.r.t 3 trainable variables: resnet_v2_50/postnorm/gamma, resnet_v2_50/postnorm/beta, resnet_v2_50/logits/biases


In [6]:
annotation = get_path_and_annotation('/media/xinje/New Volume/VOC07&12/VOC2007/train/train_OCR.txt')
train_iter = read_data(annotation,FLAGS.batch_size,input_shape=(800,1024),is_random=False,is_crop=False)

In [ ]:
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(FLAGS.log_dir,sess.graph)
    #var_list = [var for var in tf.global_variables() if ("Adam" and 'AccumGrad') not in var.name]
    var_list = tf.global_variables()
#     var_list = [var for var in tf.global_variables() if "moving" in var.name]
#     var_list += tf.trainable_variables()
    saver = tf.train.Saver(var_list=var_list,max_to_keep=5)
    sess.run(tf.global_variables_initializer())
    ckpt = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
    if ckpt:
        saver.restore(sess,ckpt)
        print('Restore from the checkpoint {0}'.format(ckpt))  
    else:
        print('Train ssd from start')
        exclude = ['resnet_v2_50' + '/logits', 'global_step','output','output2',"optimizer",'postnorm']
        variables_to_restore = tf.contrib.slim.get_variables_to_restore(exclude=exclude)
        tf.train.init_from_checkpoint('./resnet_v2_50_2017_04_14/resnet_v2_50.ckpt',
                                    {v.name.split(':')[0]: v for v in variables_to_restore})  
    for i in range(start_step,10000000):
        image_batch,annotation_batch,cls_batch = next(train_iter)
        #one hot for cls_batch
        eye = np.eye(20)
        eye = np.concatenate([eye,np.expand_dims(np.zeros_like(eye[0]),axis=0)])
        cls_batch = eye[(cls_batch-1).astype(np.int32)].astype(np.float32)
        
        matched_true_boxes,matched_true_classes,matched_true_centerness = encode_boxes(annotation_batch,cls_batch,feature_size,stride)
        feature_state = np.squeeze(matched_true_centerness!=0,axis=-1)
        
        feed_dict={inputs:image_batch,boxes:matched_true_boxes,classes:matched_true_classes,centerness:matched_true_centerness,state:feature_state}

    
        [_,train_summary,temp_loss,temp_focal_loss,temp_iou_loss,temp_centerness_loss,temp_centerness_output,temp_classes_output,temp_boxes_output] = sess.run([train_opt,merged_summary_op,total_losses,focal_losses,IOU_losses,centerness_losses,centerness_output_test,classes_output_test,boxes_output_test],feed_dict=feed_dict)
        real_step = (i-start_step)//5+start_step
        if((i-start_step)%5==0):
            print('Batch: '+str(real_step)+'   Current total loss: '+str(temp_loss)+' focal_loss: '+str(temp_focal_loss)+' IOU_loss: '+str(temp_iou_loss)+' centerness_loss: '+str(temp_centerness_loss))
#             break
            train_writer.add_summary(train_summary, real_step)
#             break
        if((i-start_step)%500==0 and i!=0):
#             save model for training
            saver.save(sess,os.path.join(FLAGS.checkpoint_dir,'FCOS-'),global_step=real_step) 
            #########################save model for inference#############################################
#             constant_graph = convert_variables_to_constants(sess, sess.graph_def, ['boxes','scores','classes'])
#             with tf.gfile.FastGFile('./model.pb', mode='wb') as f:
#                 f.write(constant_graph.SerializeToString())
            #####################################################################
            print('Model saved !!!')

INFO:tensorflow:Restoring parameters from /media/xinje/New Volume/fcos/resnet_v2_50_giou/FCOS--400
Restore from the checkpoint /media/xinje/New Volume/fcos/resnet_v2_50_giou/FCOS--400
Batch: 400   Current total loss: 7.656803 focal_loss: 2.30557 IOU_loss: 1.8226341 centerness_loss: 1.1968602
Model saved !!!
Batch: 401   Current total loss: 6.760972 focal_loss: 1.7715057 IOU_loss: 1.652236 centerness_loss: 1.0054942
Batch: 402   Current total loss: 6.0714207 focal_loss: 1.4931298 IOU_loss: 1.3684777 centerness_loss: 0.8780791
Batch: 403   Current total loss: 7.636235 focal_loss: 2.1804707 IOU_loss: 1.8795115 centerness_loss: 1.244521
Batch: 404   Current total loss: 5.7668667 focal_loss: 1.2194827 IOU_loss: 1.3945583 centerness_loss: 0.8210961
Batch: 405   Current total loss: 6.5351777 focal_loss: 1.7058067 IOU_loss: 1.5888498 centerness_loss: 0.90879476
Batch: 406   Current total loss: 5.8832483 focal_loss: 1.4490606 IOU_loss: 1.3408383 centerness_loss: 0.76162547
Batch: 407   Current 

Batch: 472   Current total loss: 6.276289 focal_loss: 1.5935102 IOU_loss: 1.4097568 centerness_loss: 0.941458
Batch: 473   Current total loss: 5.497185 focal_loss: 1.1608617 IOU_loss: 1.2251142 centerness_loss: 0.77964836
Batch: 474   Current total loss: 5.896702 focal_loss: 1.3049405 IOU_loss: 1.358567 centerness_loss: 0.9016355
Batch: 475   Current total loss: 6.186413 focal_loss: 1.519073 IOU_loss: 1.4175323 centerness_loss: 0.9182513
Batch: 476   Current total loss: 5.738877 focal_loss: 1.4285487 IOU_loss: 1.2029965 centerness_loss: 0.775778
Batch: 477   Current total loss: 6.823476 focal_loss: 1.8895437 IOU_loss: 1.558344 centerness_loss: 1.0440372
Batch: 478   Current total loss: 6.7888722 focal_loss: 2.0043654 IOU_loss: 1.424249 centerness_loss: 1.0287086
Batch: 479   Current total loss: 5.835058 focal_loss: 1.3250134 IOU_loss: 1.3481646 centerness_loss: 0.83033365
Batch: 480   Current total loss: 6.767849 focal_loss: 1.8188223 IOU_loss: 1.5824051 centerness_loss: 1.035077
Batch

Batch: 546   Current total loss: 5.1949973 focal_loss: 1.0658851 IOU_loss: 1.0375069 centerness_loss: 0.7602193
Batch: 547   Current total loss: 6.0999155 focal_loss: 1.6648856 IOU_loss: 1.2614135 centerness_loss: 0.842233
Batch: 548   Current total loss: 6.2655263 focal_loss: 1.6285467 IOU_loss: 1.4067622 centerness_loss: 0.89883643
Batch: 549   Current total loss: 5.3642883 focal_loss: 1.1090235 IOU_loss: 1.1526448 centerness_loss: 0.7712413
Batch: 550   Current total loss: 5.936485 focal_loss: 1.4168108 IOU_loss: 1.2983999 centerness_loss: 0.8898977
Batch: 551   Current total loss: 5.062745 focal_loss: 1.0577987 IOU_loss: 0.9746098 centerness_loss: 0.69896215
Batch: 552   Current total loss: 5.4616423 focal_loss: 1.1631356 IOU_loss: 1.140641 centerness_loss: 0.8264936
Batch: 553   Current total loss: 5.847885 focal_loss: 1.420889 IOU_loss: 1.3493794 centerness_loss: 0.7462474
Batch: 554   Current total loss: 7.695744 focal_loss: 2.44085 IOU_loss: 1.6858194 centerness_loss: 1.2377076

Batch: 620   Current total loss: 5.236087 focal_loss: 1.1672599 IOU_loss: 1.0169953 centerness_loss: 0.7206226
Batch: 621   Current total loss: 5.0303707 focal_loss: 0.8717855 IOU_loss: 1.0886294 centerness_loss: 0.7387484
Batch: 622   Current total loss: 7.632102 focal_loss: 2.4719381 IOU_loss: 1.5662559 centerness_loss: 1.2627028
Batch: 623   Current total loss: 5.9462404 focal_loss: 1.3976294 IOU_loss: 1.3231164 centerness_loss: 0.89429146
Batch: 624   Current total loss: 6.191327 focal_loss: 1.614565 IOU_loss: 1.2821594 centerness_loss: 0.963402
Batch: 625   Current total loss: 5.5867896 focal_loss: 1.2777604 IOU_loss: 1.1617988 centerness_loss: 0.8160325
Batch: 626   Current total loss: 5.4403105 focal_loss: 1.0739559 IOU_loss: 1.1909826 centerness_loss: 0.84417725
Batch: 627   Current total loss: 6.6075144 focal_loss: 1.903417 IOU_loss: 1.3816527 centerness_loss: 0.9912519
Batch: 628   Current total loss: 6.2314444 focal_loss: 1.709511 IOU_loss: 1.2478954 centerness_loss: 0.94284

Batch: 694   Current total loss: 6.1763444 focal_loss: 1.568075 IOU_loss: 1.1968181 centerness_loss: 1.0804166
Batch: 695   Current total loss: 5.065815 focal_loss: 1.0291382 IOU_loss: 1.0271809 centerness_loss: 0.6784632
Batch: 696   Current total loss: 5.054107 focal_loss: 0.9327269 IOU_loss: 1.0671331 centerness_loss: 0.7232168
Batch: 697   Current total loss: 5.928668 focal_loss: 1.451362 IOU_loss: 1.2255592 centerness_loss: 0.92071915
Batch: 698   Current total loss: 5.8784533 focal_loss: 1.439219 IOU_loss: 1.2077359 centerness_loss: 0.90047294
Batch: 699   Current total loss: 5.7500925 focal_loss: 1.4552975 IOU_loss: 1.1709067 centerness_loss: 0.7928657
Batch: 700   Current total loss: 6.056573 focal_loss: 1.500618 IOU_loss: 1.3454126 centerness_loss: 0.8795219
Model saved !!!
Batch: 701   Current total loss: 5.7207646 focal_loss: 1.3052834 IOU_loss: 1.1985763 centerness_loss: 0.8858868
Batch: 702   Current total loss: 6.2245636 focal_loss: 1.5824382 IOU_loss: 1.3553152 centernes

In [ ]:

# learning_rate = 1e-2
# Si = tf.Variable(tf.constant(1.0,shape=[5,1],dtype=tf.float32), trainable=True)

In [ ]:
# matched_true_boxes,matched_true_classes,matched_true_centerness = encode_boxes(annotation_batch,cls_batch,feature_size,stride)

In [ ]:
def iou_loss(y_true, y_pred,location_state,centerness):
    with tf.variable_scope("loss/iou") as scope:
        # pos location
        indices = tf.where(tf.equal(location_state, 1))
    #     print(indices.shape)
        if tf.size(indices) == 0:
            return tf.constant(0.0)
        y_regr_pred = tf.gather_nd(y_pred, indices)
    #     print(y_regr_pred.shape)
        y_regr_true = tf.gather_nd(y_true, indices)
        y_centerness_true = tf.gather_nd(centerness,indices)

        # (num_pos, )
        pred_top = y_regr_pred[:, 0]
        pred_bottom = y_regr_pred[:, 1]
        pred_left = y_regr_pred[:, 2]
        pred_right = y_regr_pred[:, 3]

        # (num_pos, )
        target_top = y_regr_true[:, 0]
        target_bottom = y_regr_true[:, 1]
        target_left = y_regr_true[:, 2]
        target_right = y_regr_true[:, 3]

        target_area = (target_left + target_right) * (target_top + target_bottom)
        pred_area = (pred_left + pred_right) * (pred_top + pred_bottom)
        w_intersect = tf.minimum(pred_left, target_left) + tf.minimum(pred_right, target_right)
        h_intersect = tf.minimum(pred_bottom, target_bottom) + tf.minimum(pred_top, target_top)

        g_w_intersect = tf.maximum(pred_left, target_left) + tf.maximum(pred_right, target_right)
        g_h_intersect = tf.maximum(pred_bottom, target_bottom) + tf.maximum(pred_top, target_top)
        ac_union = g_w_intersect * g_h_intersect
        
        area_intersect = w_intersect * h_intersect
        area_union = target_area + pred_area - area_intersect
        ious = (area_intersect) / (area_union+1e-8)
        gious = 1-ious+((ac_union-area_union)/(ac_union+1e-8))
        # (num_pos, )
        #losses = tf.reduce_sum(losses * y_centerness_true) / (tf.reduce_sum(y_centerness_true) + 1e-6)

        losses =  tf.reduce_mean(gious)
    return losses

In [ ]:
# image_batch,annotation_batch,cls_batch = next(train_iter)
# #one hot for cls_batch
# eye = np.eye(20)
# eye = np.concatenate([eye,np.expand_dims(np.zeros_like(eye[0]),axis=0)])
# cls_batch = eye[(cls_batch-1).astype(np.int32)]

# matched_true_boxes,matched_true_classes,matched_true_centerness = encode_boxes(annotation_batch,cls_batch,feature_size,stride)
# feature_state = []
# for m_t_ctn in matched_true_centerness:
#     feature_state.append((m_t_ctn!=0).astype(np.float32))

In [ ]:
# np.sum((matched_true_boxes[1]<0).astype(np.float32))

In [ ]:
# t= encode_boxes(annotation_batch,cls_batch,feature_size,stride)

In [ ]:
# for i in range(FLAGS.batch_size):
#     plt.hlines(annotation_batch[i,:,0]*800,annotation_batch[i,:,1]*1024,annotation_batch[i,:,3]*1024,'r')
#     plt.hlines(annotation_batch[i,:,2]*800,annotation_batch[i,:,1]*1024,annotation_batch[i,:,3]*1024,'r')
#     plt.vlines(annotation_batch[i,:,1]*1024,annotation_batch[i,:,0]*800,annotation_batch[i,:,2]*800,'r')
#     plt.vlines(annotation_batch[i,:,3]*1024,annotation_batch[i,:,0]*800,annotation_batch[i,:,2]*800,'r')
#     plt.imshow(image_batch[i])
#     plt.show()

In [ ]:
# annotation_batch[0]*1024

In [ ]:
# matched_true_boxes.shape

In [ ]:
# for image_index in range(FLAGS.batch_size):
#     for layer_index in range(5):
#         true_boxes = matched_true_boxes[layer_index][image_index]
#         true_classes = matched_true_classes[layer_index][image_index]
#         true_boxes = true_boxes.reshape(feature_size[layer_index][0],feature_size[layer_index][1],4)
#         true_classes = true_classes.reshape(feature_size[layer_index][0],feature_size[layer_index][1],20)
#         print(np.sum(true_boxes!=0))
#         print(true_boxes.shape)
#         for i in range(feature_size[layer_index][0]):
#             for j in range(feature_size[layer_index][1]):
#                 y_center= (i+0.5)
#                 x_center= (j+0.5)
#                 if(np.sum(true_boxes[i,j]!=0)!=0):
#                     xmin = (x_center-(true_boxes[i,j,2]*true_boxes.shape[1]))*stride[layer_index]
#                     xmax = ((true_boxes[i,j,3]*true_boxes.shape[1])+x_center)*stride[layer_index]
#                     ymin = (y_center-(true_boxes[i,j,0]*true_boxes.shape[0]))*stride[layer_index]
#                     ymax = ((true_boxes[i,j,1]*true_boxes.shape[0])+y_center)*stride[layer_index]
#         #             if((xmin>=0)and(ymin>0)and(xmax>0)and(ymax>0)):
#                     plt.hlines(ymin,xmin,xmax,'r')
#                     plt.hlines(ymax,xmin,xmax,'r')
#                     plt.vlines(xmin,ymin,ymax,'r')
#                     plt.vlines(xmax,ymin,ymax,'r')
#                     print(ymin,xmin,ymax,xmax)
#                     print(true_classes[i,j])
#                     plt.imshow(image_batch[image_index])
#                     plt.show()

In [ ]:
# for i in range(5):
#     t,b,l,r = np.split(matched_true_boxes[i],indices_or_sections=4,axis=-1)
#     area = (t+b)*(l+r)
#     print(np.sum(area>1))

In [ ]:
# np.sum(matched_true_boxes[3]>0)

In [ ]:
# pyramid_dict = resnet_base(inputs,'resnet_v1_50',is_training = True)


In [ ]:
plt.imshow(image_batch[0])

In [ ]:
# config = tf.ConfigProto(allow_soft_placement=True)
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
# config.gpu_options.allow_growth = True


In [ ]:
temp_centerness_output = temp_centerness_output.reshape(-1,1)
temp_classes_output = temp_classes_output.reshape(-1,20)
temp_boxes_output = temp_boxes_output.reshape(-1,4)

In [ ]:
mask = (temp_classes_output *temp_centerness_output)>0.5

In [ ]:
temp_boxes_output[mask[:,3]]

In [ ]:
temp_classes_output[mask[:,0]]

In [ ]:
center_list = []
for i in range(5):
    offset = np.math.floor(stride[i]/2)
    y_center_mapping = np.array([(j*stride[i]+offset) for j in range(feature_size[i][0])])/FLAGS.image_height
    x_center_mapping = np.array([(j*stride[i]+offset) for j in range(feature_size[i][1])])/FLAGS.image_width
    y_center_mapping = np.expand_dims(np.tile(np.expand_dims(y_center_mapping,axis=-1),[1,feature_size[i][1]]),axis=-1)
    x_center_mapping = np.expand_dims(np.tile(np.expand_dims(x_center_mapping,axis=0),[feature_size[i][0],1]),axis=-1)
    center = np.concatenate([y_center_mapping,x_center_mapping],axis=-1).reshape(FLAGS.batch_size,(feature_size[i][0]*feature_size[i][1]),2)
    center_list.append(center)

In [ ]:
center_list = np.squeeze((np.concatenate(center_list,axis=1)))

In [ ]:
(center_list).shape

In [ ]:
selected_center_list = np.squeeze(center_list)[mask[:,18]]

In [ ]:
ymin = (selected_center_list[:,0]-(temp_boxes_output[mask[:,18]])[:,0])*800
ymax = (selected_center_list[:,0]+(temp_boxes_output[mask[:,18]])[:,0])*800
xmin = (selected_center_list[:,1]-(temp_boxes_output[mask[:,18]])[:,1])*1024
xmax = (selected_center_list[:,1]+(temp_boxes_output[mask[:,18]])[:,1])*1024

In [ ]:
(temp_boxes_output[mask[:,18]]).shape

In [ ]:
# plt.hlines(ymin,xmin,xmax,'r')
# plt.hlines(ymax,xmin,xmax,'r')
# plt.vlines(xmin,ymin,ymax,'r')
# plt.vlines(xmax,ymin,ymax,'r')
plt.imshow(image_batch[0])
plt.show()

In [ ]:
temp_boxes_output[mask[:,18]]

In [ ]:
np.squeeze(matched_true_boxes)[mask[:,18]]

In [ ]:
t,b,l,r = np.split(boxes_3,indices_or_sections=4,axis=3)

In [ ]:
image_batch[0].shape

In [ ]:
plt.imshow(image_batch[0])

In [ ]:
boxes_3[0].shape

In [ ]:
tblr

In [ ]:
temp_classes_output = np.concatenate(temp_classes_output,axis=1).reshape(-1,20)
temp_centerness_output = np.concatenate(temp_centerness_output,axis=1).reshape(-1,1)
temp_boxes_output = np.concatenate(temp_boxes_output,axis=1).reshape(-1,4)

In [ ]:
mask = ((temp_classes_output*temp_centerness_output)>0.5)

In [ ]:
temp_boxes_output[mask[:,18]]